<h1 align='center'><b>Machine Learning Algorithms for Classification</b></h1>
<p align='center'><img src=https://estatsite.com.br/wp-content/uploads/2021/06/programming-code-coding-or-hacker-background-programming-code-icon-made-with-binary-code-digital-binary-data-and-streaming-digital-code-vector-1200x675.jpg heigth='350' width='600'></p>